In [1]:
from joblib import load
import pandas as pd

df = pd.read_csv("data/validate_all_pypi.csv", low_memory=False, keep_default_na=False)
print(len(df))
df = df[(df["num_phantom_pyfiles"] >= 0) & (df["num_maintainers"] > 0)]
print(len(df))
train_df = pd.read_csv("data/validator_dataset.csv", keep_default_na=False)
df = df[~df["name"].isin(train_df["name"])]
print(len(df))
features = [
    "num_phantom_pyfiles",
    "setup_change",
    "name_similarity",
    "tag_match",
    "num_maintainers",
    "num_maintainer_pkgs",
]
for name in ["rf"]:
    model = load(f"models/best_{name}.joblib")
    df.loc[:, name] = model.predict_proba(df[features])[:, 1]

248123
247683
228448


In [ ]:
df.sort_values("rf", ascending=False).head(100).to_csv(
    "data/validator_evaluation.csv", index=False
)

In [21]:
evaluation_df = pd.read_csv("data/validator_evaluation.csv", keep_default_na=False)
evaluation_df["label"].value_counts()

Incorrect    85
Correct      15
Name: label, dtype: int64

In [23]:
evaluation_df[evaluation_df["label"] == "Correct"]["name_similarity"].describe()

count    15.000000
mean      0.195108
std       0.043333
min       0.090909
25%       0.173750
50%       0.200000
75%       0.226496
max       0.255319
Name: name_similarity, dtype: float64